# Titanic - Machine Learning from Disaster

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from sklearn import datasets
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    roc_curve, roc_auc_score, auc,
    confusion_matrix, ConfusionMatrixDisplay,
    precision_recall_curve, f1_score, average_precision_score,
    classification_report
)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedShuffleSplit
# models simples
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# models ensemble
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import time


Carreguem la base de dades

In [ ]:
train = pd.read_csv('train.csv')
train.head()

### 1. EDA (exploratory data analysis) (1 punt)

##### Check basic info about the data set including missing value.

In [ ]:
t=train.info()

In [ ]:
d = train.describe()
d

Comptar NaNs:

In [ ]:
nan_count = train.isnull().sum()
nan_percentage = train.isnull().mean() * 100
print("Número de NaNs por columna:")
print(nan_count)
print("\nPorcentaje de NaNs por columna:")
print(nan_percentage)

#### Exploratory analysis and plots

**Plot a bar diagram to check the number of numeric entries**

From the bar diagram, it shows that there are some age entries missing as the number of count for 'Age' is less than the other counts. We can do some impute/transformation of the data to fill-up the missing entries.

In [ ]:
dT=d.T
dT.plot.bar(y='count')
plt.title("Bar plot of the count of numeric features",fontsize=10)

Check the relative size of survived and not-survived.

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived', data=train, palette='RdBu_r')

**Is there a pattern for the survivability based on sex?**

It looks like more female survived than males!

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

**Following code extracts and plots the fraction of passenger count that survived, by each class**

In [ ]:
train['Survived'] = train['Survived'].astype(float)
f_class_survived = train.groupby('Pclass')['Survived'].mean()
f_class_survived = pd.DataFrame(f_class_survived)
f_class_survived.plot.bar(y='Survived')
plt.title("Fraction of passengers survived by class",fontsize=17)

**What about any pattern related to having sibling and spouse?**

It looks like there is a weak trend that chance of survibility increased if there were more number of sibling or spouse.

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='SibSp',data=train,palette='rainbow')

In [ ]:
f_class_Age=train.groupby('Pclass')['Age'].mean()
f_class_Age = pd.DataFrame(f_class_Age)
f_class_Age.plot.bar(y='Age')
plt.title("Average age of passengers by class",fontsize=17)
plt.ylabel("Age (years)", fontsize=17)
plt.xlabel("Passenger class", fontsize=17)

### 2. Preprocessing (normalitzation, outlier removal, feature selection..) (2 punts)
* Escala de les dades.
* Impute age (by averaging) - Tractament de NaNs.
* Drop unncessary features
* Convert categorical features to dummy variables - Encodings.

#### Filling Missing Values

In [ ]:
# Crear el imputador, usando 'median' para rellenar los valores faltantes
imputer = SimpleImputer(strategy='mean')

# Ajustar y transformar los datos
train['Age'] = imputer.fit_transform(train['Age'].values.reshape(-1, 1))

# Describir el dataframe y hacer un gráfico de barras
d = train.describe()
dT = d.T
dT.plot.bar(y='count')
plt.title("Bar plot of the count of numeric features",fontsize=17)

#### Exctracting extra features

Before start dropping NaN features I found that we could take some extra features, where some of them are based of features that are about to be droppend due to their high NaN count.

In [ ]:
# hasCabin is a binary feature that says if the passanger had a Cabin or not
train['hasCabin'] = train['Cabin'].notna()

# hasFamiliar is a binary feature that says if the passanger had a familiar or not (from SibSp)
train['hasFamiliar'] = train['SibSp'] != 0

Drop unnecessary features like 'PassengerId', 'Name', 'Ticket', 'Cabin' and any other null value

In [ ]:
train.drop(['PassengerId','Name','Ticket', 'Cabin'],axis=1,inplace=True)
train.dropna(inplace=True)
train.head()

#### Convert categorial feature like 'Sex' and 'Embarked' to dummy variables and then drop the 'Sex' and 'Embarked' columns and concatenate the new dummy variables

**Use pandas 'get_dummies()' function**

In [ ]:
sex = pd.get_dummies(train['Sex'])
embark = pd.get_dummies(train['Embarked'])
train.drop(['Sex','Embarked'],axis=1,inplace=True)
train = pd.concat([train,sex,embark],axis=1)
train.head()

#### Escala de dades.

Primer mirem com queden les nostres dades.

In [ ]:
train.plot(kind='box')
plt.title('Comparació de l\'escala dels atributs')
plt.show()

Ara normalitzem:

In [ ]:
X = train.drop('Survived', axis=1).to_numpy()  # Característiques
y = train['Survived'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_cols = train.select_dtypes(include=['float64', 'int64']).columns

train[num_cols] = scaler.fit_transform(train[num_cols])

train[num_cols].plot(kind='box')
plt.title('Comparación de la escala de los atributos después de la normalización')
plt.show()

#### Polinomial features

As extra features we can add extra features which can help de model find any quadratic, cubic or basically polynomic correlation between certain features and the label, in this case we find usefull to use Numbered features such as Age and Fare.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

def create_polynomial_features(data, columns, degree):
    # Copy the DataFrame
    data_copy = data.copy()

    poly = PolynomialFeatures(degree=degree, include_bias=False)

    for c in columns:
        # Select the specific column to apply polynomial features
        selected_data = data_copy[[c]]
        poly_data = poly.fit_transform(selected_data)

        # Create column names for the new polynomial features
        poly_column_names = [f"{c}_poly_{i}" for i in range(poly_data.shape[1])]
        print(poly_column_names)

        # Add the polynomial features to the DataFrame
        data_copy[poly_column_names] = poly_data

    # Drop the original columns that were transformed
    data_copy = data_copy.drop(columns=columns)

    return data_copy

train_sq = create_polynomial_features(train, ['Age', 'Fare'], 2)
train_sq.head()


#### Preprocessing Feature Selection

##### Finding multicolineality:

In [ ]:
correlation_matrix = train.corr()

# Trobar correlacions altes
highly_correlated = (correlation_matrix > 0.8) & (correlation_matrix < 1.0)
print(highly_correlated)

##### Reducing high dimensions:

To detect high dimensions, we calculate the correlation matrix between the variables, computed in the previous cell. Variables with low or close-to-zero correlations can be considered high dimensions because they contain little relevant information. One of the indicators of the height of dimensions is the eigenspace of the correlation matrix.

In [ ]:
# Càlcul dels autovectors i autovectors
eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)

# Trobar les dimensions amb baixes contribucions
explained_variance_ratio = eigenvalues / sum(eigenvalues)
print([[var, x] for var, x in zip(explained_variance_ratio, train.columns)])

Here, as we can see, we obtain low correlation in attributes except for Pclass, Age, and SibSp. We will create a new dataframe with these variables that have a higher correlation.

In [ ]:
cutted_train = train[['Pclass', 'Age', 'SibSp']]

X = train.drop('Survived', axis=1)  # Característiques
y = train['Survived']

X_train_cutted, X_test_cutted, y_train_cutted, y_test_cutted = train_test_split(X , y, test_size=0.2, random_state=42)
scaler = StandardScaler()

X_train_cutted = scaler.fit_transform(X_train_cutted)
X_test_cutted = scaler.transform(X_test_cutted)

##### Aplicant PCA en tots les datasets

In [ ]:
from sklearn.decomposition import PCA

def plot_PCA(x, y, components=2, title='2 component PCA'):
    pca = PCA(n_components=components)
    
    principalComponents = pca.fit_transform(x)
    
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1)
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title(title, fontsize = 20)
    
    ax.scatter(principalComponents[::10,0], principalComponents[::10,1],c = y[::10], s = 40, cmap='viridis')
    ax.grid()


plot_PCA(train.drop(columns=['Survived']), train['Survived'], title='PCA amb train')
plot_PCA(train_sq.drop(columns=['Survived']), train['Survived'], title='PCA amb train_sq')
plot_PCA(cutted_train, train['Survived'], title='PCA amb cutted_train')

As we can see, with PCA, we obtain principal components with fairly scattered points in the normal training dataset.

Meanwhile, with PCA squared, we no longer obtain normalized data because we have quadratic attributes that exaggerate the values we previously normalized, creating extreme cases like the last two that are around 60000.

Finally, with the dataset trimmed to those with higher linear correlation, we obtain principal components that even seem to have some correlation, although they are still scattered between one class and another.

### 3. Metric selection (1.5 punts)

#### Model de proba

We'll use a super simple logistic regression model to have 

Funcion para mostrar el performance del modelo.

In [ ]:
def show_performance(x, y, model, title='confusion matrix', average='binary', figsize=(5, 5)):
    predictions = model.predict(x)
    acc = accuracy_score(y, predictions)
    prec = precision_score(y, predictions, average=average)
    rec = recall_score(y, predictions, average=average)
    f1 = f1_score(y, predictions, average=average)

    conf_mat = confusion_matrix(y, predictions)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

    print(f'Accuracy:{acc}')
    print(f'Precision:{prec}')
    print(f'Recall:{rec}')
    print(f'F1-score:{f1}')
    
    disp=ConfusionMatrixDisplay(conf_mat)
    disp.plot(cmap='Blues')
    plt.title(title)
    plt.show()

Funcion para mostrar la curva ROC.

In [ ]:
def generate_roc(X_test, y_test, model):
    # generate a no skill prediction (majority class)
    ns_probs = [1 for _ in range(len(y_test))]
    # predict probabilities
    lr_probs = model.predict_proba(X_test)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]

    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)

    # summarize scores
    print('Classificador sense capacitat predictiva: ROC AUC=%.3f' % (ns_auc))
    print('El nostre model: ROC AUC=%.3f' % (lr_auc))

    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    lr_fpr, lr_tpr, lr_threshold = roc_curve(y_test, lr_probs)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Sense capacitat predictiva')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='Nostre')
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

Funcion para mostrar la curva Precision/Recall.

In [ ]:
def generate_PR(X_test, y_test, model):
     # generate a no skill prediction (majority class)
    ns_probs = [1 for _ in range(len(y_test))]
    # predict probabilities
    lr_probs = model.predict_proba(X_test)
    
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]

    lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
    lr_auc = auc(lr_recall, lr_precision)
    # summarize score
    print('El nostre model té una auc=%.3f' % (lr_auc))
    # plot the precision-recall curves
    no_skill = len(y_test[y_test==1]) / len(y_test)
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='Sense capacitat predictiva')
    plt.plot(lr_recall, lr_precision, marker='.', label='Nostre')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [ ]:
X_train_MS, X_test_MS, y_train_MS, y_test_MS = train_test_split(X_train , y_train, test_size=0.2, random_state=42)

# Create and fit a logistic regression model
model = LogisticRegression()
model.fit(X_train_MS, y_train_MS)

generate_roc(X_test_MS, y_test_MS, model)
generate_PR(X_test_MS, y_test_MS, model)

show_performance(X_test_MS, y_test_MS, model)

# Make predictions on the test set
y_pred_MS = model.predict(X_test_MS)

# Gener_MSate a classification report
report = classification_report(y_test_MS, y_pred_MS)

print(report)

### 4. Model Selection amb validacio creuada (4 punts)

In [ ]:
# Creem StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=10)

def grid_search_param(model, param_grid, X_train, y_train):
    grid = GridSearchCV(model, param_grid, cv=stratified_kfold, scoring='f1')
    grid.fit(X_train, y_train)
    grid.cv_results_.keys()
    
    # Dictionary containing the parameters used to generate that score
    print(f'Best parameters: {grid.best_params_}')
    # Single best score achieved across all params (k)
    print(f'Best F1 found: {round(grid.best_score_, 5)}')
    return grid.best_estimator_

def cv_scores(model, X_train, y_train):
    start_time = time.time()
    score = cross_val_score(model, X_train, y_train, cv=stratified_kfold, scoring='f1')
    end_time = time.time()
    
    print("f1 score: ", score.mean())
    print("Time for CV: ", end_time - start_time)

#### KNN:

In [ ]:
param_grid = dict(n_neighbors=list(range(5, 20)), p=[1,2], weights=['uniform', 'distance'] )
print(f'Grid: {param_grid}')

knn = KNeighborsClassifier()
best_knn = grid_search_param(knn, param_grid, X_train, y_train)


Resultats

In [ ]:
def_knn = KNeighborsClassifier()
print("Default scores:")
cv_scores(def_knn, X_train, y_train)
print("Best estimator scores:")
cv_scores(best_knn, X_train, y_train)

Support Vector Machines (SVM):

In [ ]:
param_grid = dict(C=[0.01, 0.1, 1, 10, 100],
                    tol=[1e-2, 1e-3, 1e-4], 
                    class_weight=[None, 'balanced'])
print(f'Grid: {param_grid}')

svc = SVC(random_state=0, kernel='linear', probability=True)
best_svc = grid_search_param(svc, param_grid, X_train, y_train)

Resultats

In [ ]:
def_svc = SVC()
print("Default scores:")
cv_scores(def_svc, X_train, y_train)
print("Best estimator scores:")
cv_scores(best_svc, X_train, y_train)

AdaBoost SVC:

In [ ]:
param_grid = dict(n_estimators=list(range(10, 51, 5)), 
                  learning_rate=[0.01, 0.05, 0.1, 0.5, 1],
                  base_estimator__tol=[1e-3, 1e-4], 
                  base_estimator__class_weight=[None, 'balanced'])
print(f'Grid: {param_grid}')

ada_boost = AdaBoostClassifier(SVC(random_state=0, kernel='linear', probability=True))
best_ada_svc = grid_search_param(ada_boost, param_grid, X_train, y_train)


Resultats

In [ ]:
def_ada = AdaBoostClassifier()
print("Default scores:")
cv_scores(def_ada, X_train, y_train)
print("Best estimator scores:")
cv_scores(best_ada_svc, X_train, y_train)

RandomForest:

In [ ]:
param_grid = dict(n_estimators=list(range(75, 151, 25)), 
                  max_depth=list(range(4, 8, 1)), 
                  criterion=['gini', 'entropy'],
                  max_features=['sqrt', 'log2'])
print(f'Grid: {param_grid}')

rf = RandomForestClassifier(random_state=0)
best_rf = grid_search_param(rf, param_grid, X_train, y_train)

Resultats:

In [ ]:
def_rf = RandomForestClassifier()
print("Default scores:")
cv_scores(def_rf, X_train, y_train)
print("Best estimator scores:")
cv_scores(best_rf, X_train, y_train)

### 5.Analisi Final (1.5 punt)


In [ ]:
#knn
show_performance(X_test, y_test, best_knn)
generate_roc(X_test, y_test, best_knn)

#svc
show_performance(X_test, y_test, best_svc)
generate_roc(X_test, y_test, best_knn)

#rf
show_performance(X_test, y_test, best_rf)
generate_roc(X_test, y_test, best_rf)

#adaboost
show_performance(X_test, y_test, best_rf)
generate_roc(X_test, y_test, best_rf)